_____
<a href='http://www.bigdata.com.br'> <img src="imagens/LogoBd.png" width="300"/></a>

_____

# Uma introdução a Ciência de Dados a partir das Fake News


As *Fake News* referem-se a desinformação, ou má informação, que são disseminadas pelo boca a boca ou pela mídia tradicional e, mais recentemente, por formas digitais de comunicação, como vídeos editados, memes, anúncios não verificados e rumores propagados pelas mídias sociais. Recentemente elas se tornaram um problema sério, com o potencial de resultar em violência da multidão, suicídios etc. por causa de informações erradas circuladas nas mídias sociais.

![fake news](imagens/fake_news.jpeg)

Nesse notebook iremos explorar algoritimos do campo de **Processamento de Linguagem Natural** para reconhecer *Fake News*. Aqui, não vamos entrar muito a fundo na definição de Processamento de Linguagem Natural, mas é sulficiente saber que se trata de uma subárea do *Machine Learning*.

## Funcionalidades básicas do Jupyter Notebook
Cada célula do Jupyter roda um conjunto de linhas de código em python.
Existem dois modos de rodar uma célula:
* **Shift + Enter** para rodar a célula selecionada e ir para a próxima.
* **Alt + Enter** para rodar a célula selecionada e criar uma em seguida.

Se estiver usando o Google Colab, você pode rodar uma célula utilizando o botão *play* ao lado.


Rode a célula a seguir:

In [ ]:
print("Hello World!")

Agora rode a célula abaixo, sem criar uma nova, para importar as bibliotecas utilizadas durante o workshop:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from scipy.special import expit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from wordcloud import WordCloud,STOPWORDS
from nltk.corpus import stopwords
import re,string,unicodedata
import nltk
nltk.download('stopwords')

## Exploração de dados
Nessa parte iremos importar os dados e analisa-los, utilizando a biblioteca *pandas* exportada anteriormente:
```python
import pandas as pd
```
Iremos utilizar um banco de dados criado por pesquisadores do NILC(Núcleo Interinstitucional de Linguística Computacional), chamado [*Fake.Br Corpus*](https://github.com/roneysco/Fake.br-Corpus).

Para obtermos os dados, rode as células abaixo:

In [ ]:
# Importando o dataset de notícias verdadeiras
dataset_true_url="https://raw.githubusercontent.com/vitorhi/Workshop_bigdata/master/dataset/true.csv"
true=pd.read_csv(dataset_true_url)

In [ ]:
# Importando o dataset de notícias falsas
dataset_fake_url="https://raw.githubusercontent.com/vitorhi/Workshop_bigdata/master/dataset/fake.csv"
fake=pd.read_csv(dataset_fake_url)

Com isso, temos dois *datasets* armazenados em *dataframes* do pandas. 

Um chamado `true` e outro `fake` 

Verifique o começo do dataframe:

In [ ]:
true.head()

Faça o mesmo para o *dataframe* de notícias falsas

In [ ]:
#Insira o código aqui

Agora vamos concatenar esses *dataframes*

In [ ]:
true['category'] = 1
fake['category'] = 0

In [ ]:
df=pd.concat([true,fake], ignore_index=True)

In [ ]:
df.head()

Note que concatenamos dois *dataframes* em `df` e acrescentamos mais uma coluna. Ficando com 2 colunas: *text* e *category*.  
A coluna *category* se comporta da seguinte maneira:
* Caso a notícia em *text* seja **verdadeira**, *category* = 1
* Caso a notícia em *text* seja **falsa**, *category* = 0

Vamos obter uma coluna de `df`:

In [ ]:
df['category']

Note que a coluna *category* é muito grande, por isso não foi mostrada como um todo.   
Podemos obter uma lista de 3 linhas utilizando a sintaxe do *python*:

In [ ]:
print(df['category'][0:3])

Tente utilizar `print` para obter a notícia de índice 0 na coluna *text* em `df`:

Agora, vamos utilizar ferramentas de visualização para explorar os dados.   
A biblioteca *seaborn* (importada como **`sns`**) será usada para a visualização:

In [ ]:
sns.countplot(df['category'])

Verifica-se que há quantidades iguais de notícias falsas e verdadeiras em nossa base de dados

Agora vamos olhar algumas funções interessantes:
###  Funções interessantes

In [ ]:
# Filtros
df[df['category']==1]

Faça o mesmo para notícias falsas

In [ ]:
# Value Counts
df2 = pd.DataFrame([['Cachorro', 'Serra'],['Gato', 'Martelo'],['Cachorro', 'Martelo']], columns=['Animal', 'Ferramenta'])
df2.head()

In [ ]:
df2['Animal'].value_counts()

Mostre quantas notícias são falsas e quantas não são:

### Voltando a exploração de dados

Vamos obter as *stopwords*, para depois remove-las da nossa base de dados:

In [ ]:
stop = set(nltk.corpus.stopwords.words('portuguese'))
punctuation = list(string.punctuation)
stop.update(punctuation)

Faça o print das *stopwords* guardadas na variável `stop`:

Agora iremos remover as stopwords e URLs

In [ ]:
# Removendo URL's
def remove_links(text):
    return re.sub(r'http\S+', '', text)

# Removendo stopwords do texto
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

# Executando as duas funções anteriores
def denoise_text(text):
    text = remove_links(text)
    text = remove_stopwords(text)
    return text

# Aplicar funçao em cada notícia
df['text']=df['text'].apply(denoise_text)

Agora, vamos observar quais palavras são mais comuns em cada banco de dados, fazendo um *worldcloud*:

In [ ]:
# Para notícias verdadeiras
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800 , stopwords = stop).generate(" ".join(df[df['category'] == 1]['text']))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
# Para notícias falsas
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800 , stopwords = stop).generate(" ".join(df[df['category'] == 0].text))
plt.imshow(wc , interpolation = 'bilinear')

### Utilizando Sklearn
Vamos utilizar o Sklearn para fazer uma matriz de contagem de palavras

In [ ]:
# Serie de exemplo
pandas_string = pd.Series(['Bem vindo', 'Bem Vindo a', 'Bem Vindo a Semana da Inovação. Seja Bem vindo'])

Usando o vetorizador do Sklearn com as stop words formadas antes

In [ ]:
vectorizer = CountVectorizer(stop_words=stop)
X = vectorizer.fit_transform(pandas_string)

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
# Utilizando o método toarray() para tranforma-lá em uma array python
print(X.toarray())

Vamos realizar apenas a contagem da palavra 'Lula'

In [ ]:
vocabulary=["lula"]
vectorizer.fit_transform(vocabulary);

Realize a transdormação do dataset e a guarde na variável `X` 

Imprima os nomes das features:

Crie uma coluna chamada `NumLula` com a contagem da palavra 'lula'

## Treinando Modelo
### Classificação Logística
Rode as células a seguir:

In [ ]:
# Divisão entre dados de treino e dados de teste
x_train,x_test,y_train,y_test = train_test_split(df['NumLula'], df['category'], test_size=0.3)

# Fitting do classificador
model = linear_model.LogisticRegression(C=1e5)
model.fit(x_train.values.reshape(-1,1), y_train)


In [ ]:
prediction=model.predict(x_test.values.reshape(-1,1))

In [ ]:
# Gráfico da regressão 
plt.figure(1, figsize=(7, 6))
plt.clf()
plt.scatter(x_train.ravel(), y_train, color='black', zorder=20)
x = np.linspace(-50, 80, 300)

loss = expit(x * model.coef_ + model.intercept_).ravel()
plt.plot(x, loss, color='red', linewidth=3)

#### Matriz de confusão
Vamos introduzir o conceito de matriz de confusão:

<img src="imagens/matriz_confusao.png" width="300"/>

Faça a sua própria matriz de confusão utilizando a função `confusion_matrix()`   
Faça o *print* da *confusion matrix* para esse modelo:

Podemos calcular a precisão e outras métricas com a função `classification_report()`  
Faça o *print* da classification report para esse modelo:

Lembre-se que:
* Caso a notícia em *text* seja **verdadeira**, *category* = 1
* Caso a notícia em *text* seja **falsa**, *category* = 0

### Classificação SVC
Rode a célula a seguir:

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df['text'], df['category'], test_size=0.3, random_state=2020)

pipe = Pipeline([('vect', CountVectorizer(stop_words=stop)),
                 ('tfidf', TfidfTransformer()),
                 ('model', LinearSVC())])

model = pipe.fit(x_train, y_train)
prediction = model.predict(x_test)

Faça o *print* da `confusion_matrix` e do `classification_report`

Imprima uma notícia a seguir e depois, utilizando o método `model.predict()`, preveja se a notícia é falsa ou não:

Mais alguns exemplos:

In [ ]:
texto_falso='Dr. Ray peita Bolsonaro, chama-o de conservador fake em entrevista a Danilo Gentili e divide a direita.\n\nEste site vem avisando Jair Bolsonaro que ele deveria abandonar a pauta estatista de vez e fazer um discurso mais convincente para aquela boa parte dos liberais e conservadores do Brasil que querem se ver livres das amarras estatais.\n\nTudo bem que as pesquisas ainda dizem que a maior parte do povo é contra as privatizações, mas o índice (pouco mais de 50% do povo) é fácil de ser revertido. Ademais, Bolsonaro deveria falar para direitistas em vez de focar tanto em petistas arrependidos.\n\nRecentemente ele disse que pensaria 200 vezes antes de privatizar a Petrobrás para que ela não caia nas mãos de chineses (ou algo do tipo). Deveria ter dito: Eu garanto a privatização da Petrobrás, e também garanto que chineses não irão comprá-la. Isso não deixaria brechas. Do jeito que ele falou, parece que o suposto medo de venda aos chineses é pretexto para evitar a privatização.\n\nSeja lá como for, a direita vai ter que adotar alternativas que foquem em um estado reduzido, diminuição de impostos e venda de estatais. Além de João Amoedo, Dr. Rey está fazendo vicejar este tipo de discurso e  ainda que sua candidatura esteja em fase inicial  é complicado para Bolsonaro que apareçam pessoas de direita propondo uma visão economicamente direitista para a economia.\n\nEnfim, veja aos 32:40 Dr. Rey espinafrando Bolsonaro: Quem dá brechas não pode reclamar que os outros aproveitem, não é mesmo?'

Peguei um texto do site do G1 do dia 14/06/2020:https://g1.globo.com/sp/sao-paulo/noticia/2020/06/14/manifestantes-fazem-ato-em-defesa-da-democracia-em-sp.ghtml

E coloquei na variável a seguir:

In [ ]:
texto_verdadeiro='Manifestantes fazem ato em defesa da democracia e contra Bolsonaro em SP\n\nManifestantes realizaram um protesto na tarde deste domingo (14) na Avenida Paulista, Centro de São Paulo, em defesa da democracia, contra o presidente Jair Bolsonaro (sem partido), e contra o racismo. A concentração do ato começou às 14h em frente ao vão livre do Museu de Arte de São Paulo (Masp).\n\nO grupo seguiu em caminhada pela avenida, no sentido Paraíso, até próximo à estação Brigadeiro do Metrô, onde foi feito um discurso. O ato permaneceu pacífico até o final do trajeto. Por volta das 16h, iniciou-se a dispersão.\n\nTrês homens que portavam símbolos nazistas foram detidos. Mas, segundo a Secretaria de Segurança Pública (SSP), eles não participavam das manifestações e estariam apenas passando pela avenida quando uma mulher os denunciou para a polícia.\n\nO protesto foi realizado por integrantes das torcidas de quatro times de futebol (Corinthians, Palmeiras, São Paulo e Santos), entidades estudantis, Somos Democracia, Frente Povo Sem Medo, MTST (Movimento dos Trabalhadores Sem Teto), CMP (Central de Movimentos Populares), coletivo de mulheres, entre outros movimentos sociais.\n\nUma faixa grande com os dizeres "Fora Bolsonaro" foi estendida na avenida. Outros cartazes também traziam frases contra o fascismo e racismo.'

Outro exemplo de notícia falsa:

In [ ]:
texto_falso='NA ITÁLIA A CURA DO CORONAVIRUS É FINALMENTE ENCONTRADA.Os médicos italianos, desobedeceram a lei mundial da saúde da OMS, para não fazer autópsias nos mortos do Coronavírus e descobriram que NÃO é um VÍRUS, mas uma BACTERIA que causa a morte. Isso causa a formação de coágulos sanguíneos e causa a morte do paciente.TalA Itália derrota o chamado Covid-19, que nada mais é do que "Coagulação intravascular disseminada" (trombose).The E a maneira de combatê-lo, ou seja, sua cura, é com os “antibióticos, anti-inflamatórios e anticoagulantes”. ASPIRINA, indicando que esta doença foi mal tratada.Esta notícia sensacional para o mundo foi produzida por médicos italianos, realizando autópsias em cadáveres produzidos pelo Covid-19.OmAlgo mais, de acordo com patologistas italianos. "Os ventiladores e a unidade de terapia intensiva nunca foram necessários."Portanto, na Itália começou a mudança de protocolos, ITÁLIA A pandemia global tão conhecida é revelada e ressuscitada pela OMS, essa cura que os chineses já sabiam e não relataram PARA FAZER NEGÓCIOS.Fonte: Ministério da Saúde da Itália.COMPARTILHE QUE O MUNDO SABE QUE Fomos enganados e assassinados por nossas pessoas mais velhas !!!@ italiarevelacurardel­­­­­covid19ALERTA DOS OLHOSPasse isso para toda a sua família, vizinhança, conhecidos, amigos, colegas, colegas de trabalho ... etc. etc ... e seu ambiente em geral ...:Se eles contrairem o Covid-19 ... que não é um vírus como eles nos fizeram acreditar, mas uma bactéria ... amplificada com radiação eletromagnética 5G que também produz inflamação e hipóxia.Eles farão o seguinte:Eles vão tomar aspirina 100mg e Apronax ou Paracetamol ...Por quê? ... porque foi demonstrado que o que o Covid-19 faz é coagular o sangue, fazendo com que a pessoa desenvolva uma trombose e o sangue não flua e não oxigene o coração e os pulmões e a pessoa morra rapidamente devido a não ser capaz de respirar.Na Itália, eles estragaram o protocolo da OMS e fizeram uma autópsia em um cadáver que morreu de Covid-19 ... eles cortaram o corpo e abriram os braços e pernas e as outras seções do corpo e perceberam que as veias estavam dilatadas e coaguladas sangue e todas as veias e artérias cheias de trombos, impedindo que o sangue flua normalmente e levando oxigênio a todos os órgãos, principalmente ao cérebro, coração e pulmões e o paciente acabe morrendo,Já conhecendo esse diagnóstico, o Ministério da Saúde italiano mudou imediatamente os protocolos de tratamento Covid-19 ... e começou a administrar a seus pacientes positivos Aspirina 100mg e Apraxax ..., resultado: os pacientes começaram a se recuperar e apresentar melhorias e o Ministério of Health liberou e enviou para casa mais de 14.000 pacientes em um único dia.URGENTE: transmitir essas informações e torná-las virais, aqui em nosso país eles mentiram para nós, com essa pandemia, a única coisa que nosso presidente diz todos os dias são dados e estatísticas, mas não fornecer essas informações para salvar os cidadãos, será que também será ameaçado pelas elites? ... não sabemos, de repente todos os governos do mundo, mas a Itália quebrou a norma ... porque eles já estavam sobrecarregados e em sério caos de mortes diárias ..., agora a OMS. ... seria processado em todo o mundo por encobrir tantas mortes e o colapso das economias de muitos países do mundo ... agora entende-se por que a ordem para INCINERAR ou enterrar imediatamente os corpos sem autópsia ... e os rotulou como altamente poluente.Está em nossas mãos levar a verdade e esperar salvar muitas vidas ... ESPALHE EM TODAS AS REDES URGENTES !!!!! é por isso que o gel antibacteriano funciona e o dióxido de cloro ... Toda a PANDÊMICA é porque eles querem vacinar e chutar para assassinar as massas para controlá-las e reduzir a população mundial'